In [19]:
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

In [ ]:
# data
# politics : 0
# science : 1
# sports : 2
DATA = [ # sorry , I can't show you the data for copyright or something ...
    
]

In [3]:
# split (labels and news)
data_X = []
data_y = []
for data in DATA:
    data_X.append(data[0]) 
    data_y.append(data[1])

In [4]:
c_0 = 0
c_1 = 0
c_2 = 0
for d in data_y:
    if d == 0:
        c_0 += 1
    elif d == 1:
        c_1 += 1
    else:
        c_2 += 1
        
print('politics : ' ,c_0)
print('science : ' ,c_1)
print('sports : ' ,c_2)
print('total : ' ,c_0 + c_1 + c_2)

politics :  1000
science :  1000
sports :  1000
total :  3000


In [5]:
def flatten(data):
    for item in data:
        if hasattr(item, '__iter__'):
            for element in flatten(item):
                yield element
        else:
            yield item

flatten(data_y)
data_y = np.array(data_y)
data_y = np_utils.to_categorical(data_y)
print('data_y_shape : ', data_y.shape)


data_y_shape :  (3000, 3)


In [6]:
#sentence to words
import re
import collections

def sentence2words(sentence):
    
    stopwords =["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
                "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
                "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", 
                "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", 
                "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", 
                "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
                "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
                "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", 
                "too", "very",  "can", "will", "just",  "should", "now", "s", "t", "don", "didn", "aren", "isn", "can", "re", "ll", "ve"]
    
    sentence = sentence.lower() # lower 
    sentence = sentence.replace("\n", "") # delete the new lines
    sentence = re.sub(re.compile(r"[!-\?()' ‘’.,;/:-@[-`{-~]"), " ", sentence) # symbol to space
    sentence = sentence.split(" ") # split words with space
    sentence_words = []
    for word in sentence:
        if (re.compile(r"^.*[0-9]+.*$").fullmatch(word) is not None): # delete words with number 
            continue
        if word in stopwords: # ストップワードに含まれるものは除外
            continue
        sentence_words.append(word)        
    return sentence_words
 
# word to index
word_to_index = {}
index_to_word = {}
num_recs = 0
maxlen = 0
word_freqs = collections.Counter()
for sentence in data_X:
    sentence_words = sentence2words(sentence)
    maxlen =max(maxlen, len(sentence_words))
    for word in sentence_words:
        word_freqs[word] += 1
    num_recs += 1
    
max_features = 4000
vocab_size = min(max_features, len(word_freqs)) + 2
word_to_index = {x[0]: i+2 for i, x in
              enumerate(word_freqs.most_common(max_features))}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1
index_to_word = {v: k for k, v in word_to_index.items()}
  

    

In [7]:
# number of headlines
print('num_recs : ', num_recs)

num_recs :  3000


In [8]:
# number of words in one headline
print('maxlen : ', maxlen)

maxlen :  25


In [9]:
print('word_freqs : ' ,len(word_freqs))

word_freqs :  6832


In [10]:
# 0 = padding
# 1 = unknown
# 2 = stop words
print('word_to_index : ', len(word_to_index))

word_to_index :  4002


In [11]:
# headline to index
data_X_vec = []
for sentence in data_X:
    sentence_words = sentence2words(sentence)
    sentence_ids = []
    for word in sentence_words:
        if word not in word_to_index:
            sentence_ids.append(1)
            continue
        sentence_ids.append(word_to_index[word])
    data_X_vec.append(sentence_ids)
   
 

In [12]:
data_X = sequence.pad_sequences(data_X_vec, maxlen=20)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=0)

In [14]:
print('X_train : ', X_train.shape)
print('X_test : ', X_test.shape)
print('y_train : ', y_train.shape)
print('y_test : ', y_test.shape)

X_train :  (2400, 20)
X_test :  (600, 20)
y_train :  (2400, 3)
y_test :  (600, 3)


In [36]:
# Build model
from keras import layers

model = Sequential()
model.add(layers.Embedding(4002,64,input_length=20))
model.add(layers.Conv1D(32, 2, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 2, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(Dense(3))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 20, 64)            256128    
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 19, 32)            4128      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 3, 32)             0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 2, 32)             2080      
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 99        
_________________________________________________________________
activation_9 (Activation)    (None, 3)                 0         
Total para

In [37]:
history = model.fit(X_train, y_train, batch_size=20,
                    epochs=10,
                    validation_data=(X_test, y_test))

Train on 2400 samples, validate on 600 samples
Epoch 1/10
2400/2400 [==============================] - ETA: 0s - loss: 1.0442 - acc: 0.469 - 1s 585us/step - loss: 1.0417 - acc: 0.4704 - val_loss: 0.8294 - val_acc: 0.6350
Epoch 2/10
2400/2400 [==============================] - 1s 331us/step - loss: 0.6708 - acc: 0.7533 - val_loss: 0.5984 - val_acc: 0.7617
Epoch 3/10
2400/2400 [==============================] - 1s 326us/step - loss: 0.3608 - acc: 0.8721 - val_loss: 0.5604 - val_acc: 0.7667
Epoch 4/10
2400/2400 [==============================] - 1s 336us/step - loss: 0.2329 - acc: 0.9050 - val_loss: 0.6528 - val_acc: 0.7517
Epoch 5/10
2400/2400 [==============================] - 1s 368us/step - loss: 0.1883 - acc: 0.9154 - val_loss: 0.6451 - val_acc: 0.7683
Epoch 6/10
2400/2400 [==============================] - 1s 373us/step - loss: 0.1690 - acc: 0.9217 - val_loss: 0.6906 - val_acc: 0.7700
Epoch 7/10
2400/2400 [==============================] - 1s 369us/step - loss: 0.1565 - acc: 0.9242 

In [38]:
score, acc = model.evaluate(X_test, y_test, batch_size=10)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score, acc))


        
for i in range(10):
    idx = np.random.randint(len(X_test))
    xtest = X_test[idx].reshape(1,20)
    
    ypred = model.predict(xtest)
    predicted_class_indices=np.argmax(ypred,axis=1)
    ylabel = y_test[idx]
    ylabel = np.argmax(ylabel)
    headline = " ".join([index_to_word[x] for x in xtest[0].tolist() if x != 0])
    print('predict :', predicted_class_indices, 'label :', ylabel,'headline :', headline)
   

600/600 [==============================] - 0s 183us/step
Test score: 0.774, accuracy: 0.760
predict : [2] label : 2 headline : comeback  k keep low UNK
predict : [1] label : 1 headline : blue  UNK UNK  one ever seen
predict : [1] label : 1 headline : take look UNK UNK fighting dementia
predict : [0] label : 0 headline : full video  pelosi spoke newly elected democratic house
predict : [2] label : 2 headline : new soccer UNK tries place value UNK
predict : [0] label : 0 headline : republicans humming economy tout  trump rhetoric muddies message
predict : [2] label : 2 headline : c c  UNK UNK UNK costs         
predict : [0] label : 0 headline : justice dept  rank file tell discontent sessions approach
predict : [0] label : 1 headline : covering heritage sites threatened climate change  list got longer 
predict : [2] label : 0 headline : rick gates sought online manipulation plans israeli intelligence firm trump campaign
